# Breaking Caesar ciphers
Now we can enciper and decipher with a Caesar cipher, the next step is to write a program to automatically break enciphered messages. 

# The basic idea
The basic model is simple: we'll get the computer to try all the keys and see which is the best.

"Trying all the keys" is simple: for a Caesar cipher, we just run through the shifts 0–25.

Finding which is "best" is somewhat tricker. How do we know which is the best possible decryption? For complex ciphers, and long ciphertexts, we don't want to rely on a human to make the decision. 

## Recap: Caesar ciphers
We're just repeating the Caesar cipher implmentation from the previous notebook.

In [15]:
import string

In [16]:
def pos(letter): 
    """Return the position of a letter in the alphabet (0-25)"""
    if letter in string.ascii_lowercase:
        return ord(letter) - ord('a')
    elif letter in string.ascii_uppercase:
        return ord(letter) - ord('A')
    else:
        raise ValueError('pos requires input of {} to be an ascii letter'.format(letter))

In [17]:
def unpos(number): 
    """Return the letter in the given position in the alphabet (mod 26)"""
    return chr(number % 26 + ord('a'))

In [18]:
def caesar_encipher_letter(letter, shift):
    """Encipher a letter, given a shift amount"""
    if letter in string.ascii_letters:
        cipherletter = unpos(pos(letter) + shift)
        if letter in string.ascii_uppercase:
            return cipherletter.upper()
        else:
            return cipherletter
    else:
        return letter      

In [19]:
def caesar_encipher(message, shift):
    """Encipher a message with the Caesar cipher of given shift"""
    enciphered = ""
    for character in message:
        enciphered += caesar_encipher_letter(character, shift)
    return enciphered

In [20]:
def caesar_decipher(message, shift):
    """Decipher a message with the Caesar cipher of given shift"""
    return caesar_encipher(message, -shift)

In [21]:
cat = ''.join

In [22]:
caesar_encipher('This is a test message.', 4)

'Xlmw mw e xiwx qiwweki.'

In [23]:
caesar_decipher('Xlmw mw e xiwx qiwweki.', 4)

'This is a test message.'

# The New Stuff

# A language model
![Monkey typing](Monkey-typing.jpg) 

My approach takes the idea from the apocryphal story that an infinite number of moneys, with an infinite number of typewriters, will eventually create the complete works of Shakespeare. As the computer tries each key, generating a possible plaintext, it can score the possible plaintext by how likely it would be for a monkey, completely randomly, to generate that plaintext. 

![English letters by proportion](letter-treemap.png) 

That idea isn't that helpful when all the letters are equally likely. But if the money is using a keyboard where the keys are sized in proportion to how often they appear in English, we have something. The diagram to the right gives an idea of what such a keyboard could look like. A monkey using that keyboard, htting keys at random, will produce something like *treattlpis* than *nziuechjtk*.

That allows us to score how close a piece of text is to English. If we can work out the probability of how likely a random monkey would be to produce our possible plaintext, we can choose the key which produces the most likely plaintext.

(This model is also called a _bag of letters_ model, as it's the same as taking all the letters in the text and putting them in a bag, losing all idea of the order of letters. While we lose a lot of information with this, it's actually good enough for our purposes.)

## Finding letter probabilities
How do we find these probabilities? The easy answer is to simply read a lot of text, counting the letters. Let's use three large texts: the [complete works of Shakespeare](https://www.gutenberg.org/ebooks/100), [War and Peace](https://www.gutenberg.org/ebooks/2600), and [The Adventures of Sherlock Holmes](https://www.gutenberg.org/ebooks/48320), all from [Project Gutenberg](https://www.gutenberg.org/wiki/Main_Page).

The Python [`collections.Counter()`](https://docs.python.org/3/library/collections.html#collections.Counter) object from the standard library does a good job of counting letters for us. If we pass a `Counter` a sequence of characters, it will count them for us:

In [24]:
import collections
collections.Counter('this is some test text')

Counter({' ': 4,
         'e': 3,
         'h': 1,
         'i': 2,
         'm': 1,
         'o': 1,
         's': 4,
         't': 5,
         'x': 1})

If we `update` the `Counter`, it will update the counts for us:

In [25]:
counts = collections.Counter()
counts.update('this is some text')
counts

Counter({' ': 3,
         'e': 2,
         'h': 1,
         'i': 2,
         'm': 1,
         'o': 1,
         's': 3,
         't': 3,
         'x': 1})

In [26]:
counts.update('here is some more text')
counts

Counter({' ': 7,
         'e': 7,
         'h': 2,
         'i': 3,
         'm': 3,
         'o': 3,
         'r': 2,
         's': 5,
         't': 5,
         'x': 2})

This allows us to easily combine the counts of letters in all the texts, then write them to a file. 

In [27]:
import collections
import string

corpora = ['shakespeare.txt', 'sherlock-holmes.txt', 'war-and-peace.txt']
counts = collections.Counter()

for corpus in corpora:
    text = open(corpus, 'r').read().lower()
    counts.update(text)

letter_counts = {l: counts[l] for l in counts if l in string.ascii_lowercase}    
sorted_letters = sorted(letter_counts, key=counts.get, reverse=True)

with open('count_1l.txt', 'w') as f:
    for l in letter_counts:
        f.write("{0}\t{1}\n".format(l, counts[l]))

letter_counts

{'a': 490124,
 'b': 92919,
 'c': 141094,
 'd': 267917,
 'e': 758091,
 'f': 135318,
 'g': 117888,
 'h': 416369,
 'i': 421240,
 'j': 6679,
 'k': 54248,
 'l': 259023,
 'm': 172199,
 'n': 419374,
 'o': 504520,
 'p': 100690,
 'q': 5499,
 'r': 373599,
 's': 404473,
 't': 560576,
 'u': 190269,
 'v': 65297,
 'w': 154157,
 'x': 7414,
 'y': 143040,
 'z': 3577}

For interest, how many letters were counted?

In [28]:
sum(letter_counts.values())

6265594

At five letters per word on average, how many words is that? (Pretty printing to show the commas as separators in the number.)

In [30]:
"{:,}".format(sum(letter_counts.values()) / 5)

'1,253,118.8'

## Using the language model
Now we have the letter counts, we need to use them to score possible plaintexts. There are two stages here:
1. Convert the counts into probabilities;
2. Score a piece of text for how probable it is (by the random monkey metric).

## Converting the counts to probabilities
This just requires working out what proportion each letter is of the total counts. 

The probability of a letter occuring by chance is just the number of times that letter appears, divided by the total number of letters. 

Whe have the number of each letter in `letter_counts`: `letter_counts[letter]` is how often a letter appeared in the corpus. 

We can find the total of all the counts with `sum(letter_counts.values())`.

Division in Python is done with the `/` operator, like this:

In [31]:
12 / 4

3.0

In [32]:
12 / 5

2.4

## Your turn: find `letter_probability`s

Complete the code below. If you're stuck, you can [see the solution](letter_probability-solution.ipynb).

In [70]:
letter_probability = {}
count_sum = # Write your code here
for letter in letter_counts:
    letter_probability[letter] = # Write your code here

letter_probability

SyntaxError: invalid syntax (<ipython-input-70-d820b0f07ba3>, line 2)

In [73]:
letter_probability = {}
count_sum = sum(letter_counts.values())
for letter in letter_counts:
    letter_probability[letter] = letter_counts[letter] / count_sum

letter_probability

{'a': 0.07822466632852368,
 'b': 0.01483003846083867,
 'c': 0.022518854557125788,
 'd': 0.04276003200973443,
 'e': 0.12099267842761596,
 'f': 0.02159699463450712,
 'g': 0.0188151354843611,
 'h': 0.06645323651676122,
 'i': 0.06723065682200283,
 'j': 0.001065980336421415,
 'k': 0.00865807774969141,
 'l': 0.041340533714760326,
 'm': 0.0274832681466434,
 'n': 0.06693283988716792,
 'o': 0.08052229365643544,
 'p': 0.016070303948835497,
 'q': 0.0008776502275761883,
 'r': 0.059627068080057535,
 's': 0.06455461365674188,
 't': 0.08946893143730666,
 'u': 0.03036727244056988,
 'v': 0.010421517895988792,
 'w': 0.024603732702757314,
 'x': 0.0011832876499817894,
 'y': 0.022829439634933255,
 'z': 0.0005708955926604884}

## Score a piece of text
We can use these to find the probability of a sequence of letters by finding the probability of each letter, then multiplying them all together. What we end up with is the probability of this piece of text being typed out by the random money I introduced at the top of this notebook.

Note that the `letter_probability` dict only holds lower-case letters, so we make sure to convert the text to lowercase before tying to score it. 

In [71]:
def text_probability(text):
    prob = 1.0
    for letter in text.lower():
        if letter in letter_probability:
            prob *= letter_probability[letter]
    return prob

In [74]:
text_probability('hello')

1.1064798867415649e-06

In [75]:
text_probability('this is a longer piece of text, like what we might find in a message')

1.3917178596173123e-68

Or, in a more human-readable format:

In [76]:
'{:0.70f}'.format(text_probability('this is a longer piece of text, like what we might find in a message'))

'0.0000000000000000000000000000000000000000000000000000000000000000000139'

The final values will be very small even longer texts. There's a danger that with a long text, we'll end up with a number that's too small to represent. 

We can get around this by using the "trick" of taking logarithms of probabilities. As numbers get smaller, their logarithms get smaller, but much less quickly. This means we can still handle the probability of long texts, while still being able to see which is the most probable. 

To find the logarithm of a number, we use the `math.log10()` function.

In [48]:
import math

math.log10(1)

0.0

In [49]:
math.log10(100)

2.0

In [50]:
math.log10(0.1)

-1.0

In [51]:
math.log10(0.005)

-2.3010299956639813

When we multiply numbers, we _add_ their logarithms. Here's a multiplication:

In [52]:
0.002 * 5

0.01

Here's the logarithm of the result.

In [53]:
math.log10(0.002 * 5)

-2.0

Here are the logarithms of the two numbers.

In [54]:
math.log10(0.002), math.log10(5)

(-2.6989700043360187, 0.6989700043360189)

To multiply them together, we add their logarithms, and get the same answer as above.

In [55]:
math.log10(0.002) + math.log10(5)

-2.0

(If you're interested, we can get the original number back from the logarithm by raising 10 to a power:)

In [56]:
10 ** -2.0

0.01

Before, we multiplied the probabilities together to get the probability of the sentence. With logarithms, to get the logarithm of the probability, we add all the log probabilities of the letters. As we're only interested in which is the _most_ likely, we don't need to convert _from_ logarithms anywhere: the most likely plaintext is just the one with the highest probability, which is the highest log probability. 

## Your turn: `log_letter_probability`
Knowing what you know from above, fill the `log_letter_probability` dict with the log probabilities of the letters. 

Follow the code for filling in `letter_probability`, but take the logarithm of the fraction of letters.

If you get stuck, you can [see the solution](log_letter_probability-solution.ipynb).

In [77]:
log_letter_probability = {}

# Write your code here

log_letter_probability

{}

## Your turn: `log_text_probability()`
Using the `log_letter_probability` dict, write the `log_text_probability()` function to give the log probability of a piece of text. 

You should follow the code from `text_probability()` above, but start with a `log_probability` of zero, and _add_ the log probabilities of the letters.

If you get stuck, you can [see the solution](log_text_probability-solution.ipynb).

In [78]:
def log_text_probability(letters):
    # Write your code here

We can use that to find some log probabilities:

In [81]:
log_text_probability('hello')

-5.956056476139383

In [82]:
log_text_probability('hello there')

-11.240906451276237

Much better values!

We've ended up with a simple _model_ of the English language, which we can use to judge how likely a given piece of text is actually English. 

## Breaking Caesar ciphers
Finally, we can use `log_text_probability()` to score possible plaintexts, and hence automatically break Caesar ciphers!

We work through all 26 keys, deciphering the text with each one. For each possible plaintext, we use the `log_text_probability()` function to score how good that plaintext is. As we go through, we keep track of the best probability and the key which generated it. 

At the end, we return the best key and probability.

In [89]:
def caesar_break(message):
    """Breaks a Caesar cipher using frequency analysis"""
    # Set bad values for the best shift we've found
    best_shift = 0
    best_prob = float('-inf')
    
    # Try each key
    for shift in range(26):
        
        # Decipher the message with this key, find it's probability
        plaintext = caesar_decipher(message, shift)
        prob = log_text_probability(plaintext)
        
        # If this looks more like English than any we've seen,
        # record it as the best so far
        if prob > best_prob:
            best_prob = prob
            best_shift = shift
            
    # Return the best we found
    return best_shift, best_prob

Let's see if it works. We encipher a message, then see if `caesar_break` returns the correct key.

In [90]:
ciphertext = caesar_encipher('this is a sample message to be decrypted', 17)
ciphertext

'kyzj zj r jrdgcv dvjjrxv kf sv uvtipgkvu'

In [91]:
key, score = caesar_break(ciphertext)
key, score

(17, -41.43442244626451)

Success!

Let's try it on something larger: the ciphertext from the [National Cipher Challenge](https://www.cipherchallenge.org/) [2016 challenge 1](https://2016.cipherchallenge.org/challenges/challenge-1/).

In [86]:
c1a = open('2016-1a.ciphertext').read()
print(c1a)

PIZZG,
Q PIDM AKIVVML BPM MVKZGXBML VWBM BPM XWTQKM NWCVL WV RIUMTQI'A LMAS IVL IBBIKPML QB NWZ GWC BW TWWS IB. BPM XWTQKM LMKZGXBML QB NWZ BPMUAMTDMA (QB QA DMZG ABZIQOPBNWZEIZL WVKM GWC ZMITQAM BPIB QB PIA JMMV EZQBBMV JIKSEIZLA - QB RCAB CAMA I KIMAIZ APQNB KQXPMZ). BPM WNNQKMZ QV KPIZOM WN BPM QVDMABQOIBQWV UILM QB KTMIZ BW UM BPIB PM BPQVSA BPQA XZWDMA RIUMTQI'A LMIBP QA "RCAB" I XMZAWVIT BZIOMLG. KIZMTMAA CAM WN BPM EWZL "RCAB" MDMV QN PM QA ZQOPB, JCB Q LWV'B BPQVS PM QA. Q PIDM AXWSMV BW PMZ KWTTMIOCMA, IVL RIUMTQI LWMAV'B ABZQSM UM IA I RCUXMZ. APM EIA XZMBBG LZQDMV IVL PMZ EWZS EIA OWQVO MFBZMUMTG EMTT. IXXIZMVBTG APM EIA CVPIXXG IJWCB PMZ JWGNZQMVL TMIDQVO, JCB I YCQKS AKIV WN PMZ AMIZKP PQABWZG ACOOMABA APM EIA XZMBBG IKBQDM QV BZGQVO BW BZIKS PQU LWEV. BPM XWTQKM BPQVS BPIB APWEA PWE LMAXMZIBM APM EIA. Q BPQVS QB APWEA BPIB APM EIAV'B BPM AWZB BW OQDM CX MIAQTG.
WV WVM BPQVO Q LW IOZMM EQBP BPM XWTQKM, QB LWMAV'B AMMU DMZG TQSMTG BPIB I XPGAQKQAB EWZSQVO WV OZIDQBG EIDMA Q

In [87]:
key, score = caesar_break(c1a)
key, score

(8, -1547.4874599084396)

In [88]:
print(caesar_decipher(c1a, key))

HARRY,
I HAVE SCANNED THE ENCRYPTED NOTE THE POLICE FOUND ON JAMELIA'S DESK AND ATTACHED IT FOR YOU TO LOOK AT. THE POLICE DECRYPTED IT FOR THEMSELVES (IT IS VERY STRAIGHTFORWARD ONCE YOU REALISE THAT IT HAS BEEN WRITTEN BACKWARDS - IT JUST USES A CAESAR SHIFT CIPHER). THE OFFICER IN CHARGE OF THE INVESTIGATION MADE IT CLEAR TO ME THAT HE THINKS THIS PROVES JAMELIA'S DEATH IS "JUST" A PERSONAL TRAGEDY. CARELESS USE OF THE WORD "JUST" EVEN IF HE IS RIGHT, BUT I DON'T THINK HE IS. I HAVE SPOKEN TO HER COLLEAGUES, AND JAMELIA DOESN'T STRIKE ME AS A JUMPER. SHE WAS PRETTY DRIVEN AND HER WORK WAS GOING EXTREMELY WELL. APPARENTLY SHE WAS UNHAPPY ABOUT HER BOYFRIEND LEAVING, BUT A QUICK SCAN OF HER SEARCH HISTORY SUGGESTS SHE WAS PRETTY ACTIVE IN TRYING TO TRACK HIM DOWN. THE POLICE THINK THAT SHOWS HOW DESPERATE SHE WAS. I THINK IT SHOWS THAT SHE WASN'T THE SORT TO GIVE UP EASILY.
ON ONE THING I DO AGREE WITH THE POLICE, IT DOESN'T SEEM VERY LIKELY THAT A PHYSICIST WORKING ON GRAVITY WAVES I